In [41]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import scipy

train_df = pd.read_csv('Data/howpop_train.csv')
test_df = pd.read_csv('Data/howpop_test.csv')

corr_df = train_df.corr()

isHas = 'No'
for i in corr_df.columns:
    for j in corr_df.columns:
         if i != j and abs(corr_df[i][j]) > 0.9:
                isHas = 'Yes'
print('Corr > 0.9:' + isHas)

Corr > 0.9:No


In [45]:
publish = train_df['published'].map(lambda x: x.split('-')[0])
publish.value_counts()

2015    23452
2016    16316
2014    16180
2012    15972
2013    15537
2011    15063
2010    13091
2009    10783
2008     7743
Name: published, dtype: int64

In [58]:
features = ['author', 'flow', 'domain', 'title']
train_size = int(0.7 * train_df.shape[0])
x , y  = train_df[features], train_df['favs_lognorm']
x_test = test_df[features]
x_train, x_valid = x.iloc[:train_size, :], x.iloc[train_size:, :]
y_train, y_valid = y.iloc[:train_size], y.iloc[train_size:]

vec_title = TfidfVectorizer(min_df = 3, max_df = 0.3,   ngram_range = (1, 3))

x_train_title =  vec_title.fit_transform(x_train['title'])
x_valid_title =  vec_title.transform(x_valid['title'])
x_test_title =  vec_title.transform(x_test['title'])

len(vec_title.vocabulary_)

50624

In [61]:
vec_title.vocabulary_['python']

9065

In [83]:
vec_title_char = TfidfVectorizer(analyzer = 'char')

x_train_title_ch =  vec_title_char.fit_transform(x_train['title'])
x_valid_title_ch =  vec_title_char.transform(x_valid['title'])
x_test_title_ch =  vec_title_char.transform(x_test['title'])

len(vec_title_char.vocabulary_)

218

In [84]:
features  = features[:3]
dict_vec =  DictVectorizer()

x_train_f =  dict_vec.fit_transform(x_train[features].fillna('-').T.to_dict().values())
x_valid_f =  dict_vec.transform(x_valid[features].fillna('-').T.to_dict().values())
x_test_f =   dict_vec.transform(x_test[features].fillna('-').T.to_dict().values())

x_train_end = scipy.sparse.hstack([x_train_title, x_train_f, x_train_title_ch])
x_valid_end = scipy.sparse.hstack([x_valid_title, x_valid_f, x_valid_title_ch])
x_test_end =  scipy.sparse.hstack([x_test_title, x_test_f, x_test_title_ch])

In [87]:
ridge1 =  Ridge(alpha = 0.1, random_state = 1)
ridge2 =  Ridge(alpha = 1.0, random_state = 1)

ridge1.fit(x_train_end, y_train)
ridge2.fit(x_train_end, y_train)

tr_pred1 = ridge1.predict(x_train_end)
val_pred1 = ridge1.predict(x_valid_end)

tr_pred2 = ridge2.predict(x_train_end)
val_pred2 = ridge2.predict(x_valid_end)

print('Train1: ', mean_squared_error(y_train, tr_pred1) ,'Valid1:', mean_squared_error(y_valid, val_pred1) )
print('Train2: ',mean_squared_error(y_train, tr_pred2) ,'Valid2:', mean_squared_error(y_valid, val_pred2) )

Train1:  0.5720550548564669 Valid1: 0.8875726157845648
Train2:  0.5970371525927235 Valid2: 0.8701137380089861


In [142]:
sub = pd.read_csv('Data/sample_submission.csv', index_col = 'url')

X = scipy.sparse.vstack([x_train_end, x_valid_end])

ridge =  Ridge(alpha = 0.05, random_state = 42)
ridge.fit(X, y)
tr_preds = ridge.predict(X)
print('Train: ', mean_squared_error(y, tr_preds))

test_preds = ridge.predict(x_test_end)
ridge_sub = sub.copy()
ridge_sub['favs_lognorm'] = test_preds
ridge_sub.to_csv('ridge_baseline.csv')


Train:  0.6228333977212045
